In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
#np.random.seed(MASTER_SEED)
import tensorflow as tf
#tf.set_random_seed(MASTER_SEED)
import matplotlib.pyplot as plt
import keras

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.compat.v1.Session(config=config))
    
import time
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import *
from keras.activations import relu, sigmoid
from keras.optimizers import Adam, Nadam, RMSprop

import cv2
import math

Using TensorFlow backend.


In [2]:
doptimizer = Adam(0.0002, 0.5) # 0.0002, 0.5
goptimizer = Adam(0.00002, 0.25) # 0.0002, 0.5
# 0.0002, 0.00005

img_shape = (108, 88, 3)
latent_dim = 100

In [3]:
def build_discriminator():
    img_input = Input(shape=img_shape)
    lab_input = Input(shape=(40,))
    
    imgmodel = Sequential()
    imgmodel.add(Conv2D(64, kernel_size=8, strides=2, padding="same", input_shape=img_shape))
    imgmodel.add(BatchNormalization(momentum=0.8))
    imgmodel.add(LeakyReLU(alpha=0.2))
    imgmodel.add(Dropout(0.1))
    imgmodel.add(Conv2D(64, kernel_size=12, strides=2, padding="same"))
    imgmodel.add(BatchNormalization(momentum=0.8))
    imgmodel.add(LeakyReLU(alpha=0.2))
    imgmodel.add(Dropout(0.1))
    imgmodel.add(Conv2D(256, kernel_size=12, padding="same"))
    imgmodel.add(BatchNormalization(momentum=0.8))
    imgmodel.add(LeakyReLU(alpha=0.2))
    imgmodel.add(Dropout(0.1))
    imgmodel.add(Flatten())
    imgmodel.add(Dense(100, activation="relu"))
    imgmodel.summary()
    
    imgout = imgmodel(img_input)
    
    catout = Concatenate(axis=-1)([imgout, lab_input])
    
    finmodel = Sequential()
    finmodel.add(Dense(64, activation="relu", input_shape=[140,]))
    finmodel.add(Dropout(0.2))
    finmodel.add(Dense(1, activation="sigmoid"))
    
    validity = finmodel(catout)
    
    model = Model(inputs=[img_input, lab_input], outputs=validity)
    return model

In [4]:
def build_generator():
    model = Sequential()

    model.add(Dense(27 * 22 * 128, input_shape=(latent_dim,)))
    model.add(Activation("relu"))
    model.add(Reshape((27, 22, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=12, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=12, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(3, kernel_size=8, padding="same"))
    model.add(Activation("tanh"))

    model.summary()

    lspace = Input(shape=(latent_dim,))
    img = model(lspace)

    return Model(lspace, img)

In [5]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=doptimizer, metrics=['accuracy'])

generator = build_generator()
generator.compile(loss='mean_absolute_error', optimizer=goptimizer)

z = Input(shape=(100,))
img = generator(z)
lab = Input(shape=(40,))
discriminator.trainable = False
valid = discriminator([img, lab])

combined = Model([z, lab], valid)
combined.summary()
combined.compile(loss='binary_crossentropy', optimizer=goptimizer)







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 44, 64)        12352     
_________________________________________________________________
batch_normalization_1 (Batch (None, 54, 44, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 54, 44, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 44, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 22, 64)        589888    
_________________________________________________________________
batch_normalization_2 (Batch (None, 27, 22, 64)        256       
_____________________

In [6]:
if 'imgs' in globals():
    del imgs
imgnum = 100000 # 202599
X_train = np.empty([imgnum, 108, 88, 3])#, dtype=np.uint8)
interv = int(math.ceil(imgnum/10))
for i in range(imgnum):
    if(i%interv==0):
        print(int(i//interv), end="")
    X_train[i] = cv2.cvtColor(cv2.imread("img_mod/%06d.jpg" % (i+1)), cv2.COLOR_BGR2RGB)
X_train = (X_train / 255.0) - 0.5
print("\nDone loading images.")

Y_train = np.loadtxt("dataset/list_attr_celeba.csv", delimiter=",", skiprows=1, usecols=range(1, 41))
Y_train += 1
Y_train /= 2
Y_train = Y_train.astype(np.uint8)
print("Done loading labels.")

#X_train = np.expand_dims(X_train, )

0123456789
Done loading images.
Done loading labels.


In [7]:
def train(epochs, batch_size=128, print_interval=10, save_interval=100):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    pc = time.perf_counter()
    start_time = pc
    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        labels = Y_train[idx]
        noise = np.random.normal(0, 1, (batch_size, latent_dim-40))
        lspace = np.append(labels, noise, axis=1)
        
        gen_imgs = generator.predict(lspace)
        
        
        d_loss_fake = discriminator.train_on_batch([gen_imgs, labels], fake) # TODO (maybe): give the fake images a different set of labels
        d_loss_real = discriminator.train_on_batch([imgs, labels], valid)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        #d_loss = discriminator.train_on_batch([np.concatenate((gen_imgs, imgs)), np.concatenate((labels, labels))], np.concatenate((fake, valid)))
        
        g_loss = combined.train_on_batch([lspace, labels], valid)
        #if epoch%10==0 and d_loss[0]<0.12:
        #    if d_loss[0]<0.01:
        #        for i in range(20):
        #            generator.train_on_batch(lspace, imgs)
        #    if d_loss[0]<0.05:
        #        for i in range(5):
        #            generator.train_on_batch(lspace, imgs)
        #    generator.train_on_batch(lspace, imgs)

        if epoch%print_interval==0:
            td = time.perf_counter()-pc
            pc = time.perf_counter()
            eta = (pc-start_time) * (epochs - (epoch+1)) / (epoch+1)
            etas = eta%60
            eta //= 60
            etam = eta%60
            eta //= 60
            print ("%d [dl: %.3f, acc: %.2f%%] [gl: %f] - [tpe: %.2f, eta: %d:%02d:%02d]" % \
                   (epoch, d_loss[0], 100*d_loss[1], g_loss, td/print_interval, eta, etam, etas))
        
        if epoch%save_interval==0:
            r, c = 2, 4
            idx = np.random.randint(0, X_train.shape[0], r*c)
            imgs = X_train[idx]+0.5
            labels = Y_train[idx]
            noise = np.random.normal(0, 1, (r*c, latent_dim-40))
            lspace = np.append(labels, noise, axis=1)
            gen_imgs = generator.predict(lspace)
            
            gen_imgs = 0.5 * gen_imgs + 0.5

            fig, axs = plt.subplots(r, c)
            cnt = 0
            for i in range(0, r, 2):
                for j in range(c):
                    axs[i,j].imshow(imgs[cnt])
                    axs[i,j].axis('off')
                    axs[i+1,j].imshow(gen_imgs[cnt])
                    axs[i+1,j].axis('off')
                    cnt += 1
            fig.savefig("out_img/ep%06d.png" % epoch) # Change the %06d for different epochs
            plt.close()

In [8]:
train(200000, batch_size=64, print_interval=10, save_interval=50) #batch_size=64

0 [dl: 7.983, acc: 12.50%] [gl: 12.919127] - [tpe: 1.63, eta: 904:52:38]
10 [dl: 0.235, acc: 92.19%] [gl: 3.200589] - [tpe: 0.59, eta: 112:12:26]
20 [dl: 0.068, acc: 97.66%] [gl: 0.119219] - [tpe: 0.52, eta: 72:35:56]
30 [dl: 2.430, acc: 31.25%] [gl: 2.739892] - [tpe: 0.52, eta: 58:33:02]
40 [dl: 0.423, acc: 80.47%] [gl: 3.217920] - [tpe: 0.53, eta: 51:23:24]
50 [dl: 0.106, acc: 95.31%] [gl: 3.908872] - [tpe: 0.52, eta: 47:00:43]
60 [dl: 0.180, acc: 93.75%] [gl: 4.862253] - [tpe: 0.53, eta: 44:10:07]
70 [dl: 0.185, acc: 92.97%] [gl: 2.979220] - [tpe: 0.52, eta: 42:02:05]
80 [dl: 0.319, acc: 89.06%] [gl: 2.655877] - [tpe: 0.52, eta: 40:25:14]
90 [dl: 0.287, acc: 91.41%] [gl: 2.880342] - [tpe: 0.53, eta: 39:12:14]
100 [dl: 0.743, acc: 67.19%] [gl: 1.892739] - [tpe: 0.54, eta: 38:16:56]
110 [dl: 0.251, acc: 91.41%] [gl: 2.865605] - [tpe: 0.55, eta: 37:35:23]
120 [dl: 0.809, acc: 67.97%] [gl: 2.185385] - [tpe: 0.54, eta: 36:56:59]
130 [dl: 0.277, acc: 89.06%] [gl: 2.433976] - [tpe: 0.53, e

In [9]:
for pic in range(10):
    r, c = 2, 4
    lspace = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(lspace)

    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("out_img/FINAL%d.png" % pic)
    plt.close()